In [1]:
import geopy
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [112]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

## 1.Setting up the raw data from Calgary

In [3]:
calgary_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T'
html_data = requests.get(calgary_url).text
soup = BeautifulSoup(html_data,"html5lib") 

In [4]:
calgary_df_content = []
table = soup.find("table")
for row in table.find_all("td") :
    cell = {}
    if row.span.text=='Not assigned':
        pass
    
    else:
        borough = row.span.text.split('(')[0]
        
        if borough == 'Calgary':
            cell['PostalCode'] = row.text[:3]
            cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
            calgary_df_content.append(cell)
            
        else:
            pass

In [5]:
calgary_df1=pd.DataFrame(data=calgary_df_content)
print(calgary_df1.shape)
calgary_df1.head()

(31, 2)


,PostalCode,Neighborhood
0,T2A,"Penbrooke Meadows, Marlborough"
1,T3A,"Dalhousie, Edgemont, Hamptons, Hidden Valley"
2,T2B,"Forest Lawn, Dover, Erin Woods"
3,T3B,"Montgomery, Bowness, Silver Springs, Greenwood"
4,T2C,"Lynnwood Ridge, Ogden, Foothills Industrial, Great Plains"


In [6]:
# Had to find coordinates with https://www.latlong.net/
calgary_coordinates_url ='https://raw.githubusercontent.com/Armando12pdf/Coursera_Capstone/main/Calgary%20coordinates.csv'
calgary_coordinates_df=pd.read_csv(calgary_coordinates_url)
print(calgary_coordinates_df.shape)
calgary_coordinates_df.head()

(31, 3)


,Postal code,Latitude,Longitude
0,T2A,51.04968,-113.96432
1,T3A,51.12454,-114.14289
2,T2B,51.02533,-113.97890
3,T3B,51.08963,-114.19751
4,T2C,50.98122,-113.99786


In [7]:
#Concatenate with respective latitude and longitude
calgary_df1['Latitude'] = calgary_df1.PostalCode.map(calgary_coordinates_df.set_index('Postal code')['Latitude'].to_dict())
calgary_df1['Longitude'] = calgary_df1.PostalCode.map(calgary_coordinates_df.set_index('Postal code')['Longitude'].to_dict())
print(calgary_df1.shape)
calgary_df1.head()

(31, 4)


,PostalCode,Neighborhood,Latitude,Longitude
0,T2A,"Penbrooke Meadows, Marlborough",51.04968,-113.96432
1,T3A,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12454,-114.14289
2,T2B,"Forest Lawn, Dover, Erin Woods",51.02533,-113.97890
3,T3B,"Montgomery, Bowness, Silver Springs, Greenwood",51.08963,-114.19751
4,T2C,"Lynnwood Ridge, Ogden, Foothills Industrial, Great Plains",50.98122,-113.99786


In [8]:
#Define foursquare credentials
CLIENT_ID = 'ZZBEKKTJGWTU0HNGJ2ROO5JDHSKAXVABHUJKBBYZG1WNWQ0W' # your Foursquare ID
CLIENT_SECRET = 'JS1SDJDRRFBWX0SXVSS5GPCXVG4J3BAKY0AWJMLE2IJH4OFG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [9]:
#Function that gets venues

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
#Get venues for calgary neighborhoods
calgary_venues = getNearbyVenues(names=calgary_df1['Neighborhood'],
                                   latitudes=calgary_df1['Latitude'],
                                   longitudes=calgary_df1['Longitude'])
calgary_venues.head()

Penbrooke Meadows, Marlborough
Dalhousie, Edgemont, Hamptons, Hidden Valley
Forest Lawn, Dover, Erin Woods
Montgomery, Bowness, Silver Springs, Greenwood
Lynnwood Ridge, Ogden, Foothills Industrial, Great Plains
Rosscarrock, Westgate, Wildwood, Shaganappi, Sunalta
Bridgeland, Greenview, Zoo, YYC
Lakeview, Glendale, Killarney, Glamorgan
Inglewood, Burnsland, Chinatown, East Victoria Park, Saddledome
Hawkwood, Arbour Lake, Citadel, Ranchlands, Royal Oak, Rocky Ridge
Highfield, Burns Industrial
Discovery Ridge, Signal Hill, West Springs,Christie Park, Patterson, Cougar Ridge
Queensland, Lake Bonavista, Willow Park, Acadia
Martindale, Taradale, Falconridge, Saddle Ridge
Thorncliffe, Tuxedo Park
Sandstone, MacEwan Glen, Beddington, Harvest Hills, Coventry Hills, Panorama Hills
Brentwood, Collingwood, Nose Hill
Tuscany, Scenic Acres
Mount Pleasant, Capitol Hill, Banff Trail
Cranston, Auburn Bay, Mahogany
Kensington, Westmont, Parkdale, University
City Centre, Calgary Tower
Symons Valley
Conn

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12454,-114.14289,Tandis Online Cafe,51.124051,-114.144602,Bakery
1,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12454,-114.14289,Edgemont Construction Ltd,51.123928,-114.144313,Construction & Landscaping
2,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12454,-114.14289,Edgemont City,51.126473,-114.138997,Asian Restaurant
3,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12454,-114.14289,Friends Cappuccino Bar & Bake Shop,51.126370,-114.138676,Café
4,"Forest Lawn, Dover, Erin Woods",51.02533,-113.97890,7-Eleven,51.026558,-113.980414,Convenience Store


In [13]:
#One hot encoding of calgary venues
calgary_onehot_df = pd.get_dummies(calgary_venues[['Venue Category']], prefix="", prefix_sep="")
calgary_onehot_df.insert(loc=0, column='Neighborhood name', value=calgary_venues['Neighborhood'].values.tolist()) 
print(calgary_onehot_df.shape)
calgary_onehot_df.head()

(238, 101)


,Neighborhood name,American Restaurant,Asian Restaurant,Astrologer,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Beer Bar,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Business Service,Café,Camera Store,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Classroom,Construction & Landscaping,Convenience Store,Department Store,Diner,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Elementary School,Fast Food Restaurant,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gas Station,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym / Fitness Center,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Insurance Office,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,Music Store,New American Restaurant,Park,Pharmacy,Pier,Pizza Place,Playground,Poutine Place,Professional & Other Places,Pub,Restaurant,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shopping Mall,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Vietnamese Restaurant,Water Park,Yoga Studio
0,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Forest Lawn, Dover, Erin Woods",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
#Regroup calgary venues. This will be the data used with the ML models
calgary_grouped_df = calgary_onehot_df.groupby('Neighborhood name').mean().reset_index()
print(calgary_grouped_df.shape)
calgary_grouped_df.head()

(29, 101)


,Neighborhood name,American Restaurant,Asian Restaurant,Astrologer,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Beer Bar,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Business Service,Café,Camera Store,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Classroom,Construction & Landscaping,Convenience Store,Department Store,Diner,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Elementary School,Fast Food Restaurant,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gas Station,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym / Fitness Center,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Insurance Office,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,Music Store,New American Restaurant,Park,Pharmacy,Pier,Pizza Place,Playground,Poutine Place,Professional & Other Places,Pub,Restaurant,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shopping Mall,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Vietnamese Restaurant,Water Park,Yoga Studio
0,"Braeside, Cedarbrae, Woodbine",0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.0000,0.000000,0.0000,0.000,0.0,0.000000,0.0000,0.0,0.0000,0.0,0.000000,0.000000,0.0,0.5,0.0,0.000000,0.000,0.0,0.000000,0.0000,0.0,0.000000,0.0,0.500000,0.0,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.000000,0.0000,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0,0.000,0.0000,0.0,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.0000,0.0,0.0,0.000,0.0,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000,0.0,0.0,0.000000,0.0,0.0000
1,"Brentwood, Collingwood, Nose Hill",0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.0000,0.000000,0.0000,0.000,0.0,0.000000,0.0000,0.0,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0000,0.0,0.000000,1.0,0.000000,0.0,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.000000,0.0000,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0,0.000,0.0000,0.0,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.0000,0.0,0.0,0.000,0.0,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000,0.0,0.0,0.000000,0.0,0.0000
2,"Bridgeland, Greenview, Zoo, YYC",0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0000,0.333333,0.0,0.0000,0.333333,0.0000,0.000,0.0,0.000000,0.0000,0.0,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.0,0.333333,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.000000,0.0000,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0,0.000,0.0000,0.0,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.0000,0.0,0.0,0.000,0.0,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000,0.0,0.0,0.000000,0.0,0.0000
3,"City Centre, Calgary Tower",0.0,0.0,0.0,0.0,0.023256,0.023256,0.069767,0.0,0.0000,0.023256,0.0,0.0000,0.000000,0.0000,0.000,0.0,0.046512,0.0000,0.0,0.0000,0.0,0.023256,0.162791,0.0,0.0,0.0,0.023256,0.000,0.0,0.000000,0.0000,0.0,0.023256,0.0,0.023256,0.0,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.023256,0.0000,0.0000,0.0,0.0,0.0,0.069767,0.0,0.023256,0.023256,0.0,0.0,0.023256,0.023256,0.023256,0.0000,0.0,0.0000,0.0000,0.0,0.000,0.0000,0.0,0.023256,0.0000,0.0000,0.0000,0.0000,0.023256,0.0000,0.0,0.0,0.000,0.0,0.0000,0.0,0.046512,0.093023,0.023256,0.0000,0.00

In [15]:
# Some neighborhoods didn't get results, so they will be removed from our initial df. 
print(len(set(calgary_grouped_df['Neighborhood name'])))
print(len(set(calgary_df1['Neighborhood'])))

29
31


In [16]:
# Remove the neighborhoods in original df for which the API did provide information. 
for element in calgary_df1['Neighborhood']:
    if element in set(calgary_grouped_df['Neighborhood name']):
        pass
        
    else:
        print(element)
        calgary_df1 = calgary_df1[calgary_df1.Neighborhood != element]

print(len(set(calgary_grouped_df['Neighborhood name'])))        
print(len(calgary_df1['Neighborhood']))

Penbrooke Meadows, Marlborough
Queensland, Lake Bonavista, Willow Park, Acadia
29
29


## 2.Positive data learning with SVM

### 2.1 Prepare positively labeled raw data

In [17]:
#Postal codes were gathered manually. Their latitudes and longitudes were obtained from https://www.latlong.net/
positive_coordinates_url ='https://raw.githubusercontent.com/Armando12pdf/Coursera_Capstone/main/Positive%20locations%20coordinates.csv'
example_labs_df=pd.read_csv(positive_coordinates_url)
example_labs_df.head()

,Postal code,Latitude,Longitude
0,G1C,46.881771,-71.189369
1,G1E,46.860130,-71.194054
2,G1M,46.817230,-71.269836
3,G6W,46.757560,-71.225570
4,H1K,45.608180,-73.544520


In [18]:
#Add a neighborhood column
labs_labels =[]
for number in range(1,(example_labs_df.shape[0]+1)) :
    new_lab_number = 'example lab ' + str(number)
    labs_labels.append(new_lab_number)

example_labs_df.insert(loc=1, column='Neighborhood', value=labs_labels) 

In [19]:
example_labs_df.head()

,Postal code,Neighborhood,Latitude,Longitude
0,G1C,example lab 1,46.881771,-71.189369
1,G1E,example lab 2,46.860130,-71.194054
2,G1M,example lab 3,46.817230,-71.269836
3,G6W,example lab 4,46.757560,-71.225570
4,H1K,example lab 5,45.608180,-73.544520


In [20]:
#Get venues for the labs used as positive examples
example_labs_venues = getNearbyVenues(names=example_labs_df['Neighborhood'],
                                   latitudes=example_labs_df['Latitude'],
                                   longitudes=example_labs_df['Longitude'])

example lab 1
example lab 2
example lab 3
example lab 4
example lab 5
example lab 6
example lab 7
example lab 8
example lab 9
example lab 10
example lab 11
example lab 12
example lab 13
example lab 14
example lab 15
example lab 16
example lab 17
example lab 18
example lab 19
example lab 20
example lab 21
example lab 22
example lab 23
example lab 24
example lab 25
example lab 26
example lab 27
example lab 28
example lab 29
example lab 30
example lab 31
example lab 32
example lab 33
example lab 34
example lab 35
example lab 36
example lab 37
example lab 38
example lab 39
example lab 40
example lab 41
example lab 42
example lab 43
example lab 44
example lab 45
example lab 46
example lab 47
example lab 48
example lab 49
example lab 50
example lab 51
example lab 52
example lab 53
example lab 54
example lab 55
example lab 56
example lab 57
example lab 58
example lab 59
example lab 60
example lab 61
example lab 62
example lab 63
example lab 64
example lab 65
example lab 66
example lab 67
exam

In [21]:
example_labs_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,example lab 1,46.881771,-71.189369,Longpré Inc,46.879661,-71.190018,Home Service
1,example lab 1,46.881771,-71.189369,Assaini-Conseil S D Inc,46.883110,-71.186679,Construction & Landscaping
2,example lab 1,46.881771,-71.189369,Broue Resto,46.879427,-71.184905,American Restaurant
3,example lab 2,46.860130,-71.194054,Senor Sombrero,46.861217,-71.188514,Mexican Restaurant
4,example lab 2,46.860130,-71.194054,IGA,46.862160,-71.188334,Grocery Store


In [22]:
#One hot encoding of the example labs
labs_onehot_df = pd.get_dummies(example_labs_venues[['Venue Category']], prefix="", prefix_sep="")
labs_onehot_df.insert(loc=0, column='Neighborhood name', value=example_labs_venues['Neighborhood'].values.tolist()) 
print(labs_onehot_df.shape)
labs_onehot_df.head()

(964, 216)


,Neighborhood name,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,City,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Service,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Hockey Arena,Hockey Rink,Home Service,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Inn,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lingerie Store,Liquor Store,Luggage Store,Market,Massage Studio,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Noodle House,Office,Organic Grocery,Other Nightlife,Outdoor Supply Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Train,Train Station,Tree,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,example lab 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,example lab 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,example lab 1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,

In [23]:
#Regroup the venues of the example labs
labs_grouped_df = labs_onehot_df.groupby('Neighborhood name').mean().reset_index()
print(labs_grouped_df.shape)
labs_grouped_df.head()

(95, 216)


,Neighborhood name,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,City,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Service,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Hockey Arena,Hockey Rink,Home Service,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Inn,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lingerie Store,Liquor Store,Luggage Store,Market,Massage Studio,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Noodle House,Office,Organic Grocery,Other Nightlife,Outdoor Supply Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Train,Train Station,Tree,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,example lab 1,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,example lab 10,0.0,0.0,0

### 2.2 Train and evaluate the one-class SVM model

In [24]:
# When training a model, the features of the training data set have to match those of the target data set
# They do not match in this situation:
print('features in labs: ' + str(len(labs_grouped_df.columns)))
print('features in calgary: ' + str(len(calgary_grouped_df.columns)))

features in labs: 216
features in calgary: 101


In [25]:
#Columns in the training set that are not within the target data set, will be removed
for element in labs_grouped_df.columns.tolist():
    if element not in calgary_grouped_df.columns.tolist():
        labs_grouped_df.drop(columns=[element], inplace=True)
        
    else:
        pass

print('features in labs: ' + str(len(labs_grouped_df.columns)))
print('features in calgary: ' + str(len(calgary_grouped_df.columns)))

features in labs: 80
features in calgary: 101


In [26]:
# Repeat in other direction for features to match
# Columns in the training set that are not within the target data set, will be removed
for element in calgary_grouped_df.columns.tolist():
    if element not in labs_grouped_df.columns.tolist():
        calgary_grouped_df.drop(columns=[element], inplace=True)
        
    else:
        pass

print('features in labs: ' + str(len(labs_grouped_df.columns)))
print('features in calgary: ' + str(len(calgary_grouped_df.columns)))


features in labs: 80
features in calgary: 80


In [27]:
#Now we're ready to train our model
from sklearn.svm import OneClassSVM

In [28]:
#Split the data 
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(labs_grouped_df, test_size=0.15, random_state=4)

In [29]:
#Fit
One_class_SVM = OneClassSVM(gamma='auto').fit(X_train.loc[:, X_train.columns != 'Neighborhood name'])

In [30]:
#Test
One_c_SVM_test=One_class_SVM.predict(X_test.loc[:, X_test.columns != 'Neighborhood name'])

In [31]:
#Test how many get labelled as positive
values, counts = np.unique(One_c_SVM_test, return_counts=True)
print(values)
print(counts)
print(counts[1]/(counts[0]+counts[1]))

#The result indicates that 40% of true-positives will be labelled as negatives

[-1  1]
[6 9]
0.6


### 2.3 Applying the trained one-class SVM model

In [32]:
#Apply the one class SVM model to the calgary data frame
One_c_SVM_result=One_class_SVM.predict(calgary_grouped_df.loc[:, calgary_grouped_df.columns != 'Neighborhood name'])
One_c_SVM_result

array([-1,  1, -1,  1,  1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,  1, -1])

In [33]:
#Add results to calgary df
calgary_df1['Result'] = One_c_SVM_result
calgary_df1.head()

,PostalCode,Neighborhood,Latitude,Longitude,Result
1,T3A,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12454,-114.14289,-1
2,T2B,"Forest Lawn, Dover, Erin Woods",51.02533,-113.97890,1
3,T3B,"Montgomery, Bowness, Silver Springs, Greenwood",51.08963,-114.19751,-1
4,T2C,"Lynnwood Ridge, Ogden, Foothills Industrial, Great Plains",50.98122,-113.99786,1
5,T3C,"Rosscarrock, Westgate, Wildwood, Shaganappi, Sunalta",51.04492,-114.13070,1


In [34]:
#Get positive results only
calgary_df1_pos = calgary_df1.loc[calgary_df1['Result']==1]
calgary_df1_pos

,PostalCode,Neighborhood,Latitude,Longitude,Result
2,T2B,"Forest Lawn, Dover, Erin Woods",51.02533,-113.97890,1
4,T2C,"Lynnwood Ridge, Ogden, Foothills Industrial, Great Plains",50.98122,-113.99786,1
5,T3C,"Rosscarrock, Westgate, Wildwood, Shaganappi, Sunalta",51.04492,-114.13070,1
6,T2E,"Bridgeland, Greenview, Zoo, YYC",51.07029,-114.04284,1
8,T2G,"Inglewood, Burnsland, Chinatown, East Victoria Park, Saddledome",51.03712,-114.04642,1
17,T3L,"Tuscany, Scenic Acres",51.12323,-114.24007,1
23,T2R,"Connaught, West Victoria Park",51.04092,-114.07484,1
29,T2Y,"Millrise, Somerset, Bridlewood, Evergreen",50.91110,-114.09638,1


### 2.4 Visualization of one-class SVM model results in a map

In [35]:
# libraries for map
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



In [36]:
# create map. The latitude and longitude of toronto were found on 
# https://www.latlong.net/place/calgary-ab-canada-29104.html
svm_map = folium.Map(location=[51.049999,  -114.066666], zoom_start=10.4)

#Add markers to map
for lat, lon in zip(calgary_df1_pos['Latitude'], calgary_df1_pos['Longitude']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(svm_map)
     
svm_map

# 3 Using Positive-Unlabelled data and SVM

## 3.1 Get and prepare the unlabelled data set

In [42]:
#Put the contents in a list
unlabelled_df_content = []

In [43]:
#Define a function that gets postal codes from sites with city and neighborhood in parenthesis
def getPostalCodes1(url, city_of_interest):
    html_data = requests.get(url).text
    soup = BeautifulSoup(html_data,"html5lib") 
    table = soup.find("table")
    for row in table.find_all("td") :
        cell = {}
        if row.span.text=='Not assigned':
            pass
    
        else:
            borough = row.span.text.split('(')[0]
            if borough== city_of_interest:
                example_labs_post_codes = example_labs_df.iloc[:,0].tolist()
                PostalCode=row.text[:3]
            
                if PostalCode in example_labs_post_codes:
                    pass
            
                else:
                    cell['PostalCode'] = PostalCode
                    cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
                    unlabelled_df_content.append(cell)
    
            else:
                pass

In [44]:
url_list1=['https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_K',
          'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_R']
cities=['Ottawa', 'Winnipeg']


In [45]:
#Ottawa and Winnipeg
getPostalCodes1(url=url_list1[0], city_of_interest=cities[0])
getPostalCodes1(url=url_list1[1], city_of_interest=cities[1])

In [46]:
#Define a function that gets postal codes from sites without cities named
def getPostalCodes2(url):
    html_data = requests.get(url).text
    soup = BeautifulSoup(html_data,"html5lib") 
    table = soup.find("table")
    for row in table.find_all("td") :
        cell = {}
        if row.span.text=='Not assigned':
            pass
    
        else:
            example_labs_post_codes = example_labs_df.iloc[:,0].tolist()
            PostalCode=row.text[1:4]
            if PostalCode in example_labs_post_codes:
                pass
            
            else:
                cell['PostalCode'] = PostalCode
                cell['Neighborhood'] = row.span.text
                unlabelled_df_content.append(cell)

In [47]:
#Montreal and Laval
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_H'
getPostalCodes2(url)

In [48]:
#Define a function that gets postal codes from sites with neighborhoods in parathensis
def getPostalCodes3(url):
    html_data = requests.get(url).text
    soup = BeautifulSoup(html_data,"html5lib") 
    table = soup.find("table")
    for row in table.find_all("td") :
        cell = {}
        if row.span.text=='Not assigned':
            pass
    
        else:
            example_labs_post_codes = example_labs_df.iloc[:,0].tolist()
            PostalCode=row.text[1:4]
            if PostalCode in example_labs_post_codes:
                pass
            
            else:
                cell['PostalCode'] = PostalCode
                cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')   
                unlabelled_df_content.append(cell)

In [49]:
#Toronto
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
getPostalCodes3(url)

In [50]:
#Put together in a dataframe
unlabelled_df=pd.DataFrame(data=unlabelled_df_content)
print(unlabelled_df.shape)
print(len(set(unlabelled_df.iloc[:,0].tolist())))
unlabelled_df.head()

(256, 2)
256


,PostalCode,Neighborhood
0,K2A,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood"
1,K4A,Fallingbrook
2,K1B,"Blackburn Hamlet, Pine View, Sheffield Glen"
3,K2B,"Britannia,Whitehaven, Bayshore, Pinecrest"
4,K4B,Navan


In [51]:
#Unlabelled coordinates
unlabelled_coordinates_url ='https://raw.githubusercontent.com/Armando12pdf/Coursera_Capstone/main/Unlabelled%20locations%20coordinates.csv'
unlabelled_coordinates_df=pd.read_csv(unlabelled_coordinates_url)
unlabelled_coordinates_df.head()

,Postal code,Latitude,Longitude
0,K2A,45.38025,-75.76138
1,K4A,45.46734,-75.47799
2,K1B,45.42042,-75.59603
3,K2B,45.36172,-75.78945
4,K4B,45.41413,-75.40364


In [52]:
#Concatenate with respective latitude and longitude
unlabelled_df['Latitude'] = unlabelled_df.PostalCode.map(unlabelled_coordinates_df.set_index('Postal code')['Latitude'].to_dict())
unlabelled_df['Longitude'] = unlabelled_df.PostalCode.map(unlabelled_coordinates_df.set_index('Postal code')['Longitude'].to_dict())
print(unlabelled_df.shape)
unlabelled_df.head()

(256, 4)


,PostalCode,Neighborhood,Latitude,Longitude
0,K2A,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood",45.38025,-75.76138
1,K4A,Fallingbrook,45.46734,-75.47799
2,K1B,"Blackburn Hamlet, Pine View, Sheffield Glen",45.42042,-75.59603
3,K2B,"Britannia,Whitehaven, Bayshore, Pinecrest",45.36172,-75.78945
4,K4B,Navan,45.41413,-75.40364


In [53]:
#Get venues for unlabelled neighborhoods
unlabelled_venues = getNearbyVenues(names=unlabelled_df['Neighborhood'],
                                   latitudes=unlabelled_df['Latitude'],
                                   longitudes=unlabelled_df['Longitude'])

Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood
Fallingbrook
Blackburn Hamlet, Pine View, Sheffield Glen
Britannia,Whitehaven, Bayshore, Pinecrest
Navan
Queensway, Copeland Park, Central Park, Bel Air,Carleton Heights
Cumberland
Queenswood
Eastern Nepean: Fisher Heights/ Parkwood Hills, Borden Farm, Pine Glen
Centrepointe, Meadowlands, City View, Craig Henry, Tangelwood, Grenfell Glen, Davidson Heights
Bells Corners, Arlington Woods/Redwood, Qualicum, Crystal Beach
Beacon Hill, Cyrville, Carson Grove
Barrhaven
Beaverbrook, South March
Katimavik-Hazeldean, Glen Cairn
Rockcliffe Park, New Edinburgh
Bridlewood
Manotick
Downtown
Greely
Dalhousie Ward
Fallowfield Village, Cedarhill Estates, Orchard Estates
The Glebe, Old Ottawa South, Old Ottawa East, Carleton University, Dow's Lake area
Stittsville
Blossom Park, Greenboro, Leitrim, Findlay Creek
Marchwood
Terry Fox, Palladium
Chapel Hill South, Blackburn
North March
South Gloucester
Civic Hospital, Island Park, Hintonburg, Me

In [54]:
print(unlabelled_venues.shape)
unlabelled_venues.head()

(4985, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood",45.38025,-75.76138,Tillbury Park,45.378744,-75.759590,Park
1,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood",45.38025,-75.76138,Westboro Badminton Club,45.383524,-75.759266,Sports Club
2,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood",45.38025,-75.76138,McKellar Park,45.382858,-75.765691,Park
3,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood",45.38025,-75.76138,Jademark Technologies,45.383348,-75.756911,Electronics Store
4,Fallingbrook,45.46734,-75.47799,L&F Montego Place,45.467689,-75.475943,Gourmet Shop


In [55]:
#One hot encoding of the unlabelled neighborhoods
unlab_onehot_df = pd.get_dummies(unlabelled_venues[['Venue Category']], prefix="", prefix_sep="")
unlab_onehot_df.insert(loc=0, column='Neighborhood name', value=unlabelled_venues['Neighborhood'].values.tolist()) 
print(unlab_onehot_df.shape)
unlab_onehot_df.head()

(4985, 344)


,Neighborhood name,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Sake B

In [56]:
#Regroup the venues of the example labs
unlab_grouped_df = unlab_onehot_df.groupby('Neighborhood name').mean().reset_index()
print(unlab_grouped_df.shape)
unlab_grouped_df.head()

(246, 344)


,Neighborhood name,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Sake B

## 3.2 Prepare the PU data set

In [57]:
#The postivie data set, unlabelled data, and calgary data set do not match in features
print('Before matching:')
print('Features in unlabelled data set: ' + str(len(unlab_grouped_df.columns)))
print('Features in positive data set: ' + str(len(labs_grouped_df.columns)))
print('Features in calgary data set: ' + str(len(calgary_grouped_df.columns)))

Before matching:
Features in unlabelled data set: 344
Features in positive data set: 80
Features in calgary data set: 80


In [58]:
#Make the postivie data set, unlabelled data, and calgary data set match in features
all_features =[]
all_features = unlab_grouped_df.columns.tolist() + labs_grouped_df.columns.tolist() + calgary_grouped_df.columns.tolist()

print('Total number of features is: ' + str(len(calgary_grouped_df.columns)+len(labs_grouped_df.columns)+len(unlab_grouped_df.columns)))
print('Number of features put together in a single list: ' + str(len(all_features)))


Total number of features is: 504
Number of features put together in a single list: 504


In [59]:
features_values, features_counts = np.unique(all_features, return_counts=True)

features_dict = {'features':features_values, 'counts':features_counts}
features_df1 = pd.DataFrame(features_dict)
features_df1.iloc[1:7,:]

,features,counts
1,Accessories Store,1
2,Adult Boutique,1
3,Afghan Restaurant,1
4,African Restaurant,1
5,Airport,1
6,Airport Lounge,1


In [60]:
features_df2=features_df1.loc[features_df1['counts']==3]
features_df2.head()

,features,counts
9,American Restaurant,3
17,Asian Restaurant,3
18,Athletics & Sports,3
24,Bakery,3
25,Bank,3


In [96]:
common_features=features_df2['features'].values.tolist()
unlab_grouped_df2 = unlab_grouped_df[common_features]
labs_grouped_df2 = labs_grouped_df[common_features]
calgary_grouped_df2 = calgary_grouped_df[common_features]

print('Before matching:')
print('Features in unlabelled data set: ' + str(len(unlab_grouped_df2.columns)))
print('Features in positive data set: ' + str(len(labs_grouped_df2.columns)))
print('Features in calgary data set: ' + str(len(calgary_grouped_df2.columns)))

Before matching:
Features in unlabelled data set: 78
Features in positive data set: 78
Features in calgary data set: 78


In [176]:
#Add labels column to positive and unlabelled data sets
# '1' is used for the positive data set
# '0' is used for the unlabelled data set

labs_grouped_df2['Label'] = np.ones(labs_grouped_df2.shape[0])
unlab_grouped_df2['Label'] = np.zeros(unlab_grouped_df2.shape[0])


#labs_grouped_df2['Label']=[1.0]*(labs_grouped_df2.shape[0])
#unlab_grouped_df2['Label'] = [0.0]*(unlab_grouped_df2.shape[0])

<ipython-input-176-7e41f78a40e3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labs_grouped_df2['Label'] = np.ones(labs_grouped_df2.shape[0])
<ipython-input-176-7e41f78a40e3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unlab_grouped_df2['Label'] = np.zeros(unlab_grouped_df2.shape[0])


In [177]:
#Concatenate data sets

print('Shape of positive data set: ' + str(labs_grouped_df2.shape))
print('Shape of unlabelled data set: ' + str(unlab_grouped_df2.shape))

pu_df = pd.concat([labs_grouped_df2, unlab_grouped_df2], axis=0, ignore_index=True)

print('Shape of PU data set: ' + str(pu_df.shape))

Shape of positive data set: (95, 79)
Shape of unlabelled data set: (246, 79)
Shape of PU data set: (341, 79)


## 3.3 Train and evaluate an SVM model with the PU data

In [173]:
# Use pulearn library
!pip install pulearn

In [178]:
# Prepare the data for splitting
X_pu = pu_df.drop(columns=['Label', 'Neighborhood name'])
y_pu = pu_df['Label'].tolist()

print('Shape of PU set: ' + str(pu_df.shape))
print('Shape of PU features set: ' + str(X_pu.shape))
print('Length of PU labels set: ' + str(len(y_pu)))



Shape of PU set: (341, 79)
Shape of PU features set: (341, 77)
Length of PU labels set: 341


In [179]:
# Split the data
X_train_pu, X_test_pu, y_train_pu, y_test_pu = train_test_split(X_pu, y_pu, test_size=0.2, random_state=4)


In [180]:
# Train the model
from pulearn import ElkanotoPuClassifier
from sklearn.svm import SVC

pu_svc = SVC(C=10, kernel='rbf', gamma=0.4, probability=True)
pu_estimator = ElkanotoPuClassifier(estimator=pu_svc, hold_out_ratio=0.2)
pu_estimator.fit(X_train_pu, y_train_pu)

KeyError: "None of [Int64Index([137, 170, 125,  36, 132, 122, 243, 130, 180,  17, 263, 225, 266,\n            229, 190,  22,  10, 154, 102,  64, 204, 247, 100, 250,  93, 123,\n            254, 193,  91, 268, 203,  15, 166, 181, 262, 165, 215,   5,  77,\n             29, 239,  80,  38, 188, 217,  83, 158, 109,  85,  60,  24,   4,\n             20,  42,  54],\n           dtype='int64')] are in the [columns]"

In [181]:
from pulearn import BaggingPuClassifier

pu_svc2 = SVC(C=10, kernel='rbf', gamma=0.4, probability=True)
pu_estimator2 = BaggingPuClassifier(base_estimator=s, n_estimators=15)

NameError: name 's' is not defined

In [182]:
pu_estimator.fit(X_train_pu, y_train_pu)

KeyError: "None of [Int64Index([ 70,  66, 217, 270, 145, 199, 234, 269, 209,  17, 108, 107, 124,\n             21, 119,  85,   1, 158, 229, 110, 170,  83, 179, 151,  34,  58,\n             75, 236, 139,  81,  26, 246,  35, 256,  42,  23,  86,   2, 227,\n            200, 189,  77, 111,   0, 125, 134, 265, 182, 204, 164, 122, 213,\n            155, 118, 255],\n           dtype='int64')] are in the [columns]"